In [224]:
import backtesting as bt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn import metrics
from backtesting import Backtest, Strategy

df = pd.read_csv("btcusdt15m-150622.csv")
X = df[predictors]  # predictor
y = df.UP_DOWN  # response

predictors = ["CMO", "MINUSDM", "PLUSDM", "CLOSE","CLOSEL1", "CLOSEL2", "CLOSEL3", "3O", "CMB"]
# Split test & training set
X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=1)


all = ["CMO", "MINUSDM", "PLUSDM", "CLOSE", "CLOSEL1", "CLOSEL2", "CLOSEL3", "3O", "CMB", "Open", "Close", "High", "Low"]
X_test_all = df[all].iloc[-2000:]
y_test_all = df["UP_DOWN"].iloc[-2000:]

# Normalize data
scaler = StandardScaler().fit(X_train[predictors[0:7]])
X_train[predictors[0:7]] = scaler.transform(X_train[predictors[0:7]])
X_test[predictors[0:7]] = scaler.transform(X_test[predictors[0:7]])
X_test_all[predictors[0:7]] = scaler.transform(X_test_all[predictors[0:7]])


# Fit Model using training data
svm = SVC(kernel="rbf", C=1)
svm = svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
print("Accuracy: ", (cnf_matrix[0][0]+cnf_matrix[1][1])/(cnf_matrix[0][0]+cnf_matrix[0][1]+cnf_matrix[1][1]+cnf_matrix[1][0]))

Accuracy:  0.6933920704845815


In [208]:
test_df = X_test_all.join(y_test_all)


,CMO,MINUSDM,PLUSDM,CLOSE,CLOSEL1,CLOSEL2,CLOSEL3,3O,CMB
9349,-0.999332,0.760933,-0.642946,-1.242773,-1.246534,-1.196201,-1.162984,0,0


In [236]:
import warnings
warnings.filterwarnings('ignore')

class SVM(Strategy):

    def init(self):        

        # Prepare empty, all-NaN forecast indicator
        high, low, close = self.data.High, self.data.Low, self.data.Close
        self.x = 1
        self.prev_forecast = 0

        
    def next(self):
        
        X = X_test_all[predictors].iloc[:self.x].values

        forecast = svm.predict([X[-1]])[0]

        if forecast == 1 and not self.position.is_long:
            self.buy(size=.2)
        elif forecast == 0 and not self.position.is_short:
            self.sell(size=.2)
            

        self.x += 1

bt = Backtest(test_df, SVM, commission=.0002, cash=100000, margin=0.5)
bt.run()

Start                                  9349.0
End                                   11348.0
Duration                               1999.0
Exposure Time [%]                        99.9
Equity Final [$]                119518.757764
Equity Peak [$]                 119600.210502
Return [%]                          19.518758
Buy & Hold Return [%]                0.456795
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.131743
Avg. Drawdown [%]                    -0.09602
Max. Drawdown Duration                  217.0
Avg. Drawdown Duration               10.56213
# Trades                                563.0
Win Rate [%]                        59.147425
Best Trade [%]                       5.804534
Worst Trade [%]                      -3.22821
Avg. Trade [%]                    

In [223]:
bt.plot()

Row(id='6992', ...)